Course Instructor: Bernd Neumayr, JKU

# UE04- SPARQL Updata and RDF Datasets

Complete the **8 tasks (1 point per task)** in the `3. SPARQL Update` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- Your solution in executable form: your solutions for SemAI.jar will make use of the default grap. In this notebook you have to transform your solutions according to the workaround exemplified in V04_SPARQL_Update.ipynb
- After executing the update request, print a serizalization of the dataset in TriG format.  

**Task 9 (2 points)**  is to develop a nice visualization of RDF datasets using `visualize_graph_pyvis` from UE02 as a starting point. The requirements are as follows:
- Each named graph must be represented as an independent graph. This means, for example, that :Jane in :JanesGraph is a different node than :Jane in :BillsGraph. There are no edges between nodes in different graphs.
- It is not strictly necessary to draw a box around each named graph, as seen in the slides. The different named graphs should simply be visually distinguishable and not overlap.
- If not all nodes within a named graph are connected, make sure in the visualization that the named graph still forms a coherent visual unit in some way.

## Preparations

In [ ]:
# Install required packages
!pip install -q rdflib     # comment to avoid re-install with every re-run

### Imports and Functions 

We are re-using the sparql_select function. 

In [ ]:
# Imports
import pandas as pd
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace


# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      


# Task 1
Sie beginnen mit einem leeren Dataset. Fügen Sie in den Default Graph Statements ein, die sagen, dass :Peter der Autor von :G1 ist, und :Mary Autor von :G2.

In [ ]:
g = rdflib.Graph().parse(format="trig",data="""
  @prefix : <http://example.org/> .""")

g.update("""
INSERT DATA {
  :G1 :author :Mary .
  :G2 :author :Peter .
};
""")

print(g.serialize(format="trig"))

In [ ]:
ds = rdflib.Dataset()

ds.parse(format="trig", data="""
@prefix : <http://example.org/> .

:main {
  :G1 :author :Mary .
  :G2 :author :Peter .
}
""")

print(ds.serialize(format="trig"))

# Task 2
Schreiben Sie { :Mary :knows :Peter, :John, :Mary. } in den Named Graph :G1 und { :Peter :knows :Mary. :John :knows :Mary. } in den Named Graph :G2.

In [ ]:
ds.update("""
INSERT DATA { 
  GRAPH :G1{
    :Mary :knows :Mary;
    :knows :John;
    :knows :Peter .
  }
  GRAPH :G2{
    :John :knows :Mary .
    :Peter :knows :Mary .
  }
}
""")

print(ds.serialize(format="trig"))

# Task 3
Fragen Sie mittels INSERT-WHERE die :knows-Beziehungen aus :G2 ab und fügen deren inverse :knownBy-Beziehungen in den Default-Graph ein.
Ihr Update Request darf nicht enthalten: [Mary, Peter, John]

In [ ]:
ds.update("""
INSERT { 
  GRAPH :main {
    ?o :knownBy ?s . 
  } 
}
WHERE {
  GRAPH :G2 { 
    ?s :knows ?o. 
  } 
}
""")

print(ds.serialize(format="trig"))

# Task 4
Löschen Sie mittels DELETE-WHERE alle :knownBy-Beziehungen aus dem Default-Graph.
Ihr Update Request darf nicht enthalten: [Mary, Peter, John]

In [ ]:
ds.update("""
DELETE { 
  GRAPH :main {
    ?s :knownBy ?o . 
  }
}
WHERE { 
  GRAPH :main {
    ?s :knownBy ?o .
  }
}

""")

print(ds.serialize(format="trig"))

# Task 5
Ermitteln Sie mittels INSERT-WHERE zu jedem Named-Graph dessen Anzahl an Statements mit der Property :knows und schreiben Sie diese in den Default Graph.
Ihr Update Request darf nicht enthalten: [G1, G2]

In [ ]:
ds.update("""
INSERT { 
  GRAPH :main {
    ?g :knowsCount ?n .
    # filter not working?
    # FILTER (?n != 0) .
  }
}
WHERE {
  GRAPH ?g { 
    SELECT (COUNT(*) AS ?n)
    WHERE{ 
       ?p :knows ?o.
    }
  } 
}

""")

print(ds.serialize(format="trig"))

# Task 6
Ermitteln Sie mittels INSERT-WHERE die Anzahl an Named Graphs und schreiben Sie sie in den Default Graph.
Ihr Update Request darf nicht enthalten: [2]

In [ ]:
ds.update("""
INSERT { 
  GRAPH :main {
    :ds :graphCount ?n .
  }
}
WHERE {
 SELECT (COUNT(DISTINCT ?g) AS ?n)
 WHERE{ 
   GRAPH ?g {
    ?s ?p ?o .
   }
   FILTER (?g != :main)
 }
}

""")

print(ds.serialize(format="trig"))

# Task 7
Verschieben Sie mittels DELETE-INSERT-WHERE alle Metadaten zu Named Graphs (also Statements die einen Named Graph als Subjekt haben) in den entsprechenden Named Graph.
Ihr Update Request darf nicht enthalten: [G1, G2]

In [ ]:
ds.update("""
DELETE {
  GRAPH :main {
    ?metadata  ?p ?o . 
  }
}
INSERT {
  GRAPH ?metadata {?metadata ?p ?o . }
}
WHERE
{ 
  GRAPH :main {
    ?metadata ?p ?o .
    FILTER(?metadata != :ds)
  }
}


""")

print(ds.serialize(format="trig"))

semai_query = """
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

DELETE {?metadata  ?p ?o . }
INSERT {GRAPH ?metadata {?metadata ?p ?o . }}
WHERE
{ 
  ?metadata ?p ?o .
  FILTER(?metadata != :ds)
}
"""

# Task 8
Schreiben Sie in jeden Named Graph ein Statement, dass der Autor des jeweiligen Named Graphs die :Susi kennt und aktualisieren Sie mit dem selben UpdateRequest den knowsCount.
Ihr Update Request darf nicht enthalten: [Peter, Mary]

In [ ]:
ds.update("""
DELETE { GRAPH ?graph  {?graph  :knowsCount ?count . }}
INSERT {
  GRAPH ?graph {
    ?author :knows :Susi .
    ?graph :knowsCount ?newCount .
  }
}
WHERE {
  GRAPH ?graph {
    ?graph :author ?author;
       :knowsCount ?count .
    BIND(?count + 1 AS ?newCount)
  }
}
""")

print(ds.serialize(format="trig"))

semai_query = """
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

DELETE { GRAPH ?graph  {?graph  :knowsCount ?count . }}
INSERT {
  GRAPH ?graph {
    ?author :knows :Susi .
    ?graph :knowsCount ?newCount .
  }
}
WHERE {
  GRAPH ?graph {
    ?graph :author ?author;
       :knowsCount ?count .
    BIND(?count + 1 AS ?newCount)
  }
}
"""